In [1]:
import pandas as pd
import json
import pickle

In [2]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

In [3]:
train_df=   pd.read_csv('TeleoSplitSlack/train.csv')
test_df=    pd.read_csv('TeleoSplitSlack/test.csv')
val_df=   pd.read_csv('TeleoSplitSlack/val.csv')
total_df = pd.concat([train_df, test_df, val_df])

In [4]:
families = list(train_df['family'].unique())
order = list(train_df['order'].unique())
print(type(families))
classes_dict = {'order': order, 'family': families}
print(len(classes_dict['order']))
print(len(classes_dict['family']))

<class 'list'>
78
378


In [ ]:
with open('data/data_set_multi/classes_dict.pkl', 'wb') as f:
    pickle.dump(classes_dict, f)
    

In [ ]:
pickle.load(open(r"C:\Users\Auguste\Desktop\ADNe\bertax_training_2\data\data_set_multi\classes_dict.pkl", 'rb'))

In [5]:
id_dict = ncbi.get_name_translator(total_df['family'].tolist())
for key in id_dict:
    
    id_dict[key] = id_dict[key][0]

In [6]:
train_df['family_id'] = train_df['family'].map(id_dict)
test_df['family_id'] = test_df['family'].map(id_dict)
val_df['family_id'] = val_df['family'].map(id_dict)
total_df['family_id'] = total_df['family'].map(id_dict)


In [7]:
train=pd.DataFrame(columns=['x', 'y','id','family'])
train['x']=train_df['sequence']
train['y']= train_df['family']
train['tax_id']=train_df["family_id"]
train['family']=train_df['family']
train['order']=train_df['order']

test=pd.DataFrame(columns=['x', 'y','id','family'])
test['x']=test_df['sequence']
test['y']= test_df['family']
test['tax_id']=test_df["family_id"]
test['family']=test_df['family']
test['order']=test_df['order']


val=pd.DataFrame(columns=['x', 'y','id','family'])
val['x']=val_df['sequence']
val['y']= val_df['family']
val['tax_id']=val_df["family_id"]
val['family']=val_df['family']
val['order']=val_df['order']

train.to_csv("data/data_set_multi/train.tsv", sep="\t", index=False)
test.to_csv("data/data_set_multi/test.tsv", sep="\t", index=False)
val.to_csv("data/data_set_multi/val.tsv", sep="\t", index=False)


OSError: Cannot save file into a non-existent directory: 'data\data_set_multi'

In [11]:
for f in total_df['family'].unique():

    with open(f'data/fasta/{f}.fa', 'x') as fasta:

        for i, row in total_df[total_df['family'] == f].iterrows():
            fasta.write(f'>{row["family_id"]} {i}\n{row["sequence"]}\n')

In [9]:


for f in test_df['family'].unique():

    with open(f'data/test/{f}.fa', 'x') as fasta:

        for i, row in test_df[test_df['family'] == f].iterrows():
            fasta.write(f'>{row["family_id"][0]} {i}\n{row["sequence"]}\n')

for f in val_df['family'].unique():

    with open(f'data/val/{f}.fa', 'x') as fasta:

        for i, row in val_df[val_df['family'] == f].iterrows():
            fasta.write(f'>{row["family_id"][0]} {i}\n{row["sequence"]}\n')

## to fragment


In [28]:
import json
import argparse
from Bio.SeqIO import parse
import os



    

In [15]:
for fasta in os.listdir('data/fasta/'):

    records = list(parse(f'data/fasta/{fasta}', 'fasta'))

    fragments = [str(r.seq) for r in records]

    species_list = [r.id for r in records]

    json.dump(fragments, open(f'data/json/{fasta[:-3]}_fragments.json', 'w'))
    with open(f'data/json/{fasta[:-3]}_species_picked.txt', 'w') as f:
        f.write('\n'.join(species_list))

In [15]:



for fasta in os.listdir('data/test/'):

    records = list(parse(f'data/test/{fasta}', 'fasta'))

    fragments = [str(r.seq) for r in records]

    species_list = [r.id for r in records]

    json.dump(fragments, open(f'data/json_test/{fasta[:-3]}_fragments.json', 'w'))
    with open(f'data/json_test/{fasta[:-3]}_species_picked.txt', 'w') as f:
        f.write('\n'.join(species_list))

for fasta in os.listdir('data/val/'):

    records = list(parse(f'data/val/{fasta}', 'fasta'))

    fragments = [str(r.seq) for r in records]

    species_list = [r.id for r in records]

    json.dump(fragments, open(f'data/json_val/{fasta[:-3]}_fragments.json', 'w'))
    with open(f'data/json_val/{fasta[:-3]}_species_picked.txt', 'w') as f:
        f.write('\n'.join(species_list))

# Read pediction

In [29]:
import json
import os
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
import pandas as pd



In [52]:
predicted = pickle.load(open("C:\\Users\\Auguste\\Desktop\\ADNe\\bertax_training_2\\data\\out\\test_predictions.pkl", 'rb'))

In [53]:

y_true, y_pred = predicted["data"]

In [55]:



acc = balanced_accuracy_score(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
print(f" acc:", acc)
print(pd.crosstab(np.argmax(np.array(y_true), axis=1), np.argmax(np.array(y_pred), axis=1),
                    rownames=['True'], colnames=['Predicted'], margins=True))
result = predicted['metrics']
metrics_names = predicted['metrics_names']


 acc: 0.42306600539696926
Predicted  0   1   3  4  5  6  8  10  12  14  ...  348  349  351  353  358  \
True                                          ...                            
0          6   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
1          0   6   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
2          0   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
3          0   0  14  0  0  0  0   0   0   0  ...    0    0    0    0    0   
4          0   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
...       ..  ..  .. .. .. .. ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   
374        0   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
375        0   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
376        0   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
377        0   0   0  0  0  0  0   0   0   0  ...    0    0    0    0    0   
All        6  12  14  2  1  2  1   5  

c:\Users\Auguste\anaconda3\envs\b_utils\Lib\site-packages\sklearn\metrics\_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [56]:
acc_dict_order = {}
for i , true in enumerate(np.argmax(y_true, axis=-1)) :
    if true in acc_dict_order:
        acc_dict_order[true][0] += 1
    else:
        acc_dict_order[true] = [1, 0]
    if true == np.argmax(y_pred, axis=-1)[i]:
        acc_dict_order[true][1] += 1
print("family only : miccro_acc", sum([acc_dict_order[key][1]/acc_dict_order[key][0] for key in acc_dict_order])/len(acc_dict_order))
print('family only : maccro_acc', sum([acc_dict_order[key][1] for key in acc_dict_order])/sum([acc_dict_order[key][0] for key in acc_dict_order]))

family only : miccro_acc 0.42306600539696926
family only : maccro_acc 0.7724750277469479


In [51]:
acc_dict_order = {}
for i , true in enumerate(np.argmax(y_true[0], axis=-1)) :
    if true in acc_dict_order:
        acc_dict_order[true][0] += 1
    else:
        acc_dict_order[true] = [1, 0]
    if true == np.argmax(y_pred[0], axis=-1)[i]:
        acc_dict_order[true][1] += 1
print("order : miccro_acc", sum([acc_dict_order[key][1]/acc_dict_order[key][0] for key in acc_dict_order])/len(acc_dict_order))
print('order : maccro_acc', sum([acc_dict_order[key][1] for key in acc_dict_order])/sum([acc_dict_order[key][0] for key in acc_dict_order]))

acc_dict_family = {}
for i , true in enumerate(np.argmax(y_true[1], axis=-1)) :
    if true in acc_dict_family:
        acc_dict_family[true][0] += 1
    else:
        acc_dict_family[true] = [1, 0]
    if true == np.argmax(y_pred[1], axis=-1)[i]:
        acc_dict_family[true][1] += 1
print("family : miccro_acc", sum([acc_dict_family[key][1]/acc_dict_family[key][0] for key in acc_dict_family])/len(acc_dict_family))
print('family : maccro_acc', sum([acc_dict_family[key][1] for key in acc_dict_family])/sum([acc_dict_family[key][0] for key in acc_dict_family]))

order : miccro_acc 0.6879205574334791
order : maccro_acc 0.8129855715871254
family : miccro_acc 0.5539489052330553
family : maccro_acc 0.7497225305216426


In [21]:
acc_dict

{np.int64(66): [18, 17],
 np.int64(111): [62, 59],
 np.int64(310): [86, 86],
 np.int64(234): [2, 2],
 np.int64(301): [25, 23],
 np.int64(67): [248, 248],
 np.int64(54): [10, 1],
 np.int64(107): [1, 1],
 np.int64(57): [1, 0],
 np.int64(56): [12, 5],
 np.int64(159): [78, 69],
 np.int64(281): [22, 22],
 np.int64(323): [28, 20],
 np.int64(244): [5, 2],
 np.int64(139): [2, 2],
 np.int64(27): [25, 14],
 np.int64(232): [15, 15],
 np.int64(19): [7, 7],
 np.int64(348): [14, 14],
 np.int64(1): [14, 6],
 np.int64(303): [1, 1],
 np.int64(297): [1, 0],
 np.int64(204): [5, 5],
 np.int64(24): [13, 6],
 np.int64(197): [78, 74],
 np.int64(102): [55, 55],
 np.int64(206): [11, 9],
 np.int64(316): [30, 29],
 np.int64(43): [1, 0],
 np.int64(160): [20, 15],
 np.int64(199): [3, 3],
 np.int64(140): [11, 11],
 np.int64(308): [9, 4],
 np.int64(3): [14, 14],
 np.int64(377): [7, 0],
 np.int64(40): [1, 0],
 np.int64(150): [8, 7],
 np.int64(345): [3, 0],
 np.int64(373): [19, 16],
 np.int64(189): [32, 29],
 np.int64